In [1]:
from datetime import datetime
from strategy.stratfact import StratFact
from backtester.backtester import Backtester
from strategy.financialpredict import FinancialPredict

In [2]:
start = datetime(2020,1,1)
end = datetime(2021,1,1)

In [ ]:
# rp = StratFact.create_strat(start,end,"rolling_percent",params= {"timeframe":"daily"
#                 ,"requirement":5
#                 ,"days":7
#                 ,"value":True
#                 ,"currency":"crypto"})
# pr = StratFact.create_strat(start,end,"progress_report",params={"timeframe":"quarterly"
#                     ,"requirement":5})
start_date = datetime(2015,1,1)
end_date = datetime(2021,1,1)
fp = FinancialPredict(start,end)

In [ ]:
fp

In [4]:
# rpt = Backtester.equity_timeseries_backtest(start,end,5,rp)

In [5]:
# pr = Backtester.equity_timeseries_backtest(start,end,5,rp)

,date,adjclose,ticker,seat,sell_date,sell_price,timeframe,requirement,days,value,currency
0,2020-02-25,247.817596,ETH,0,2020-07-22,260.208476,daily,5,7,True,crypto
1,2020-08-21,389.126343,ETH,0,2020-08-30,408.582660,daily,5,7,True,crypto
2,2020-09-03,10245.296875,BTC,0,2020-09-15,10757.561719,daily,5,7,True,crypto
3,2020-09-21,341.786072,ETH,0,2020-09-29,358.875376,daily,5,7,True,crypto
4,2020-11-26,518.801147,ETH,0,2020-11-29,544.741204,daily,5,7,True,crypto
5,2020-12-08,554.827759,ETH,0,2020-12-13,582.569147,daily,5,7,True,crypto
6,2020-12-23,583.714600,ETH,0,2020-12-25,612.900330,daily,5,7,True,crypto
7,2021-01-11,1090.145386,ETH,0,2021-01-14,1144.652655,daily,5,7,True,crypto
8,2020-02-26,8820.522461,BTC,1,2020-05-06,9261.548584,daily,5,7,True,crypto
9,2020-05-10,8756.430664,BTC,1,2020-05-13,9194.252197,daily,5,7,True,crypto


In [ ]:
fpb = Backtester.equity_timeseries_backtest(start,end,5,fp)